In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
spark = SparkSession.builder.master("local[*]").appName('spark_intermunicipal_sc').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 13:33:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
tb = spark.read.parquet("ref/parquet")
tb.createOrReplaceTempView("tb")

dicionario_secoes = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema", "true").option("encoding", "utf-8").csv("dicionario/dicionario_secoes_stripped.csv")
dicionario_secoes.createOrReplaceTempView("dicionario_secoes")

dicionario_linhas = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema", "true").option("encoding", "utf-8").csv("dicionario/dicionario_linhas_stripped.csv")
dicionario_linhas.createOrReplaceTempView("dicionario_linhas")

22/04/05 13:33:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
# verificação dos arquivos lidos

print(f"\
      tb = {tb.count()} linhas | {len(tb.columns)} colunas\n\
      dicionario_secoes = {dicionario_secoes.count()} linhas | {len(dicionario_secoes.columns)} colunas\n\
      dicionario_linhas = {dicionario_linhas.count()} linhas | {len(dicionario_linhas.columns)} colunas\
      ")

      tb = 615285 linhas | 53 colunas
      dicionario_secoes = 1710 linhas | 2 colunas
      dicionario_linhas = 242 linhas | 2 colunas      


In [5]:
tb.printSchema()

root
 |-- ano: long (nullable = true)
 |-- codigo: double (nullable = true)
 |-- des_localidade_id: double (nullable = true)
 |-- des_localidade_nome: string (nullable = true)
 |-- des_localidade_uf: double (nullable = true)
 |-- des_municipio_id: double (nullable = true)
 |-- des_municipio_nome: string (nullable = true)
 |-- empresa: string (nullable = true)
 |-- empresa_cnpj: double (nullable = true)
 |-- empresa_situacao: double (nullable = true)
 |-- empresa_tipo: double (nullable = true)
 |-- ida_idoso_desconto: double (nullable = true)
 |-- ida_idoso_gratis: double (nullable = true)
 |-- ida_jovem_desconto: double (nullable = true)
 |-- ida_jovem_gratis: double (nullable = true)
 |-- ida_pagantes: double (nullable = true)
 |-- ida_passelivre: double (nullable = true)
 |-- km: string (nullable = true)
 |-- km_total: double (nullable = true)
 |-- linha: double (nullable = true)
 |-- linha_id: double (nullable = true)
 |-- lugares_idas: double (nullable = true)
 |-- lugares_voltas: 

In [6]:
df_join = spark.sql("""

SELECT
tb.ano AS `ano`,
tb.empresa AS `empresa`,
servico_ambito AS `modalidade`,
CONCAT(prefixo, ' - ', CONCAT(linha_ori.sig, '-', linha_des.sig)) AS linha,
prefixo,
CONCAT(linha_ori.sig, '-', linha_des.sig) AS sig_linha_ori_des,
CONCAT(secao_ori.sig, '-', secao_des.sig) AS sig_secao_ori_des,
km,
pax_total AS `passageiros_total`,
tb.ori_municipio_nome AS sie_linha_ori,
tb.des_municipio_nome AS sie_linha_des,
tb.ori_localidade_nome AS sie_secao_ori,
tb.des_localidade_nome AS sie_secao_des,
linha_ori.sig AS sig_linha_ori,
linha_des.sig AS sig_linha_des,
secao_ori.sig AS sig_secao_ori,
secao_des.sig AS sig_secao_des
FROM tb
LEFT JOIN dicionario_secoes AS secao_ori ON tb.ori_localidade_nome = secao_ori.sie
LEFT JOIN dicionario_secoes AS secao_des ON tb.des_localidade_nome = secao_des.sie
LEFT JOIN dicionario_linhas AS linha_ori ON tb.ori_municipio_nome = linha_ori.sie
LEFT JOIN dicionario_linhas AS linha_des ON tb.des_municipio_nome = linha_des.sie
ORDER BY `ano`, `empresa`, `prefixo`

""")

In [7]:
print((df_join.count(), len(df_join.columns)))

(615285, 17)


In [8]:
df_join.show(5, False)

+----+-----------------------------+----------+------------------------------+-------+----------------------+-------------------------------+------+-----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+
|ano |empresa                      |modalidade|linha                         |prefixo|sig_linha_ori_des     |sig_secao_ori_des              |km    |passageiros_total|sie_linha_ori|sie_linha_des|sie_secao_ori|sie_secao_des|sig_linha_ori|sig_linha_des|sig_secao_ori|sig_secao_des       |
+----+-----------------------------+----------+------------------------------+-------+----------------------+-------------------------------+------+-----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+
|2000|1-EMP STO ANJO DA GUARDA LTDA|RODOVIARIO|149-0 - CRICIÚMASC-JOINVILLESC|149-0  |CRICIÚMASC-JOINVILLESC|CRICIÚMASC-JOINVILLESC         |3

# Gravação

In [9]:
df_join.write.option("header", "True").option("encoding", "utf-8").parquet("ref_join/parquet/multifile/")
# df_join.write.option("header", "True").option("encoding", "utf-8").parquet("ref_join/parquet/singlefile/")
df_join.write.option("delimiter", ";").option("header", "True").option("encoding", "utf-8").csv("ref_join/csv/multifile/")
# df_join.write.option("delimiter", ";").option("header", "True").option("encoding", "utf-8").csv("ref_join/csv/singlefile/")
# df_join.write.partitionBy("ano", "empresa").option("delimiter", ";").option("header", "true").option("encoding", "utf-8").csv("./intermunicipal_sc/ref_join/")

22/04/05 13:34:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


# Teste

In [10]:
leitura_joined = spark.read.option('delimiter', ';').option('encoding', 'utf-8').option('header', 'true').parquet("ref_join/parquet/multifile/")

def teste_qualidade(df, n_colunas, n_linhas):
    if (len(df.columns) == n_colunas) == True and (int(df.count()) == n_linhas) == True:
        print("OK")
    else:
        print("FAILED")

teste_qualidade(leitura_joined, 17, 615285)

OK


In [12]:
leitura_joined.createOrReplaceTempView("qd")

In [13]:
ano = 2019
cidade_sig = "FLORIANÓPOLISSC"
modalidade = "RODOVIARIO"
df_html = spark.sql(f"""

SELECT ano, empresa, modalidade, prefixo AS linha_prefixo, CONCAT(sie_linha_ori, '-', sie_linha_des) AS linha_descricao, CONCAT(sie_secao_ori, '-', sie_secao_des) AS linha_secao, sig_secao_ori_des AS linha_secao_sig, passageiros_total FROM qd WHERE passageiros_total > 0 AND ano = {ano} AND modalidade = '{modalidade}' AND sig_secao_ori_des LIKE "%{cidade_sig}%";

""").show(10, False)

# df_html.toPandas().to_html('teste.html', index = False)

+----+-----------------------------+----------+-------------+-------------------------+---------------------------+-----------------------------+-----------------+
|ano |empresa                      |modalidade|linha_prefixo|linha_descricao          |linha_secao                |linha_secao_sig              |passageiros_total|
+----+-----------------------------+----------+-------------+-------------------------+---------------------------+-----------------------------+-----------------+
|2019|1-EMP STO ANJO DA GUARDA LTDA|RODOVIARIO|1116-0       |PAULO LOPES-FLORIANOPOLIS|PAULO LOPES-FLORIANOPOLIS  |PAULO LOPESSC-FLORIANÓPOLISSC|5                |
|2019|1-EMP STO ANJO DA GUARDA LTDA|RODOVIARIO|1116-0       |PAULO LOPES-FLORIANOPOLIS|MORRETES-FLORIANOPOLIS     |PALHOÇASC-FLORIANÓPOLISSC    |4                |
|2019|1-EMP STO ANJO DA GUARDA LTDA|RODOVIARIO|1116-0       |PAULO LOPES-FLORIANOPOLIS|MACIAMBU-FLORIANOPOLIS     |PALHOÇASC-FLORIANÓPOLISSC    |4                |
|2019|1-EMP STO 

In [14]:
leitura_joined.printSchema()

root
 |-- ano: long (nullable = true)
 |-- empresa: string (nullable = true)
 |-- modalidade: string (nullable = true)
 |-- linha: string (nullable = true)
 |-- prefixo: string (nullable = true)
 |-- sig_linha_ori_des: string (nullable = true)
 |-- sig_secao_ori_des: string (nullable = true)
 |-- km: string (nullable = true)
 |-- passageiros_total: string (nullable = true)
 |-- sie_linha_ori: string (nullable = true)
 |-- sie_linha_des: string (nullable = true)
 |-- sie_secao_ori: string (nullable = true)
 |-- sie_secao_des: string (nullable = true)
 |-- sig_linha_ori: string (nullable = true)
 |-- sig_linha_des: string (nullable = true)
 |-- sig_secao_ori: string (nullable = true)
 |-- sig_secao_des: string (nullable = true)



# Exemplos

## Seleção com atributos usando spark.sql
* algumas colunas
* 2019 como ano-base
* empresas de código 23 e 1505 (Reunidas)
* movimentação de passageiros superior a 10.000
* modalidade (âmbito) do serviço rodoviário
* ordenação decrescente do total de passageiros

In [ ]:
spark.sql("""

SELECT ano AS Ano, empresa AS Empresa, CONCAT(ori_municipio_nome, '-', des_municipio_nome) AS Linha, ori_des_localidade_nome AS `Origem-destino`, pax_total AS `Total de passageiros`
FROM tb
WHERE ano = 2019
AND (empresa LIKE '23%' OR empresa LIKE '1505%')
AND pax_total > 10000
AND servico_ambito = 'RODOVIARIO'
ORDER BY pax_total DESC

""").show(10, False)

## Seleção com atributos usando select do DataFrame
* algumas colunas

In [ ]:
tb.select(
    "ano",
    "empresa",
    "prefixo",
    concat_ws("-", "ori_municipio_nome", "des_municipio_nome").alias("linha_ori_des_sie"),
    concat_ws("-", "ori_localidade_nome", "des_localidade_nome").alias("secao_ori_des_sie"),
    "pax_total"
).show(10, False)

## Seleção com atributos usando select do DataFrame
* algumas colunas
* concatenação de dados

In [ ]:
tb.select(
    "ano",
    "empresa",
    struct("prefixo", concat_ws("-", "ori_municipio_nome", "des_municipio_nome").alias("linha_ori_des_sie")).alias("linha"),
    concat_ws("-", "ori_localidade_nome", "des_localidade_nome").alias("secao_ori_des_sie"),
    "pax_total"
).show(10, False)

##### _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

In [ ]:
schema = StructType([
    StructField("ano", LongType(), True),
    StructField("codigo", StringType(), True),
    StructField("des_localidade_id", StringType(), True),
    StructField("des_localidade_nome", StringType(), True),
    StructField("des_localidade_uf", StringType(), True),
    StructField("des_municipio_id", StringType(), True),
    StructField("des_municipio_nome", StringType(), True),
    StructField("empresa", StringType(), True),
    StructField("empresa_cnpj", StringType(), True),
    StructField("empresa_situacao", StringType(), True),
    StructField("empresa_tipo", StringType(), True),
    StructField("ida_idoso_desconto", StringType(), True),
    StructField("ida_idoso_gratis", StringType(), True),
    StructField("ida_jovem_desconto", StringType(), True),
    StructField("ida_jovem_gratis", StringType(), True),
    StructField("ida_pagantes", StringType(), True),
    StructField("ida_passelivre", StringType(), True),
    StructField("km", StringType(), True),
    StructField("km_total", StringType(), True),
    StructField("linha", StringType(), True),
    StructField("linha_id", StringType(), True),
    StructField("lugares_idas", StringType(), True),
    StructField("lugares_voltas", StringType(), True),
    StructField("mes", StringType(), True),
    StructField("ori_des_localidade_nome", StringType(), True),
    StructField("ori_localidade_id", StringType(), True),
    StructField("ori_localidade_nome", StringType(), True),
    StructField("ori_localidade_uf", StringType(), True),
    StructField("ori_municipio_id", StringType(), True),
    StructField("ori_municipio_nome", StringType(), True),
    StructField("pax_gratis_descontos", StringType(), True),
    StructField("pax_idoso_desconto", StringType(), True),
    StructField("pax_idoso_gratis", StringType(), True),
    StructField("pax_jovem_desconto", StringType(), True),
    StructField("pax_jovem_gratis", StringType(), True),
    StructField("pax_pagantes", StringType(), True),
    StructField("pax_passelivre", StringType(), True),
    StructField("pax_total", StringType(), True),
    StructField("prefixo", StringType(), True),
    StructField("secao_id", StringType(), True),
    StructField("sequencial", StringType(), True),
    StructField("servico_ambito", StringType(), True),
    StructField("servico_tipo", StringType(), True),
    StructField("sisdap_fim", StringType(), True),
    StructField("sisdap_inicio", StringType(), True),
    StructField("viagem_idas", StringType(), True),
    StructField("viagem_voltas", StringType(), True),
    StructField("volta_idoso_desconto", StringType(), True),
    StructField("volta_idoso_gratis", StringType(), True),
    StructField("volta_jovem_desconto", StringType(), True),
    StructField("volta_jovem_gratis", StringType(), True),
    StructField("volta_pagantes", StringType(), True),
    StructField("volta_passelivre", StringType(), True)])